In [ ]:
import numpy as np
from sklearn.decomposition import NMF
from sklearn.metrics import mean_absolute_error

import pandas as pd
from scipy import linalg as la
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype
from scipy.sparse import linalg as spla

In [3]:
#Benjamin's path: path = "/Users/Armen/Desktop/SpringDataProject/"
#Ben C's path: path = "/Users/benchristensen/Desktop/Recommneder_System_Project/ml-20m/"
path = "/Users/Armen/Desktop/SpringDataProject/"
r = pd.read_csv(path+"ratings.csv",nrows=1000000)

In [4]:
m = pd.read_csv(path+'movies.csv',index_col = 0)

In [5]:
#get only those movies with more than 17 ratings (cutting out ~50% of movies)
merged = r.merge(r.groupby("movieId").size().reset_index(name='count'),
                 how='right', on='movieId')
df = merged[merged["count"]>17].sort_values(['userId', 'movieId'])

#create a matrix from the ratings list
user_c = CategoricalDtype(sorted(df.userId.unique()), ordered=True)
movie_c = CategoricalDtype(sorted(df.movieId.unique()), ordered=True)
row = df.userId.astype(user_c).cat.codes
col = df.movieId.astype(movie_c).cat.codes

#this matrix is movies x users
sparse_matrix = csr_matrix((df['rating'], (row, col)), \
                           shape=(user_c.categories.size, movie_c.categories.size)).T

#we can't deal with these NaN's, so we're lazy and just replace them all with 3's
dfs = pd.SparseDataFrame(sparse_matrix, \
                     index=movie_c.categories, \
                     columns=user_c.categories, \
                     default_fill_value=2.5)

In [6]:
model = NMF(n_components = 20,init='random')
W = model.fit_transform(dfs)
H = model.components_
X = W@H

In [7]:
def get_topics(W):
    """Each column of W contains a 'relevance score' for how much each movie 
        is related to the 20 "types" we found. This function gets the top
        # of movies in that group and returns them together as strings
    """
    top = 10
    data = []
    for i in range(W.shape[1]):
        scores = W[:,i]
        positions = np.argsort(scores)[-top:]
        
        movie_ids = np.array(movie_c.categories)[positions]
        strings = []
        for ind,j in enumerate(movie_ids):
            strings.append(m.loc[j].title + m.loc[j].genres)
        data.append(strings)
        
    return pd.DataFrame(np.array(data).T)

In [8]:
print(get_topics(W))

                                                  0   \
0                        Karate Kid, The (1984)Drama   
1    King Kong (1933)Action|Adventure|Fantasy|Horror   
2  Wallace & Gromit: The Best of Aardman Animatio...   
3                    Kalifornia (1993)Drama|Thriller   
4                          Analyze This (1999)Comedy   
5                        Richard III (1995)Drama|War   
6  Antz (1998)Adventure|Animation|Children|Comedy...   
7    Sneakers (1992)Action|Comedy|Crime|Drama|Sci-Fi   
8  Dracula (Bram Stoker's Dracula) (1992)Fantasy|...   
9  Who Framed Roger Rabbit? (1988)Adventure|Anima...   

                                                  1   \
0  Spy Who Loved Me, The (1977)Action|Adventure|T...   
1  Man with the Golden Gun, The (1974)Action|Adve...   
2         Goldfinger (1964)Action|Adventure|Thriller   
3                        Richard III (1995)Drama|War   
4  Outlaw Josey Wales, The (1976)Action|Adventure...   
5                  High Plains Drifter (1973)We

In [9]:
score = mean_absolute_error(y_true=dfs, y_pred=X)
score

0.057216855495346206

# Our Very Own NMF Algorithm

In [ ]:
def nmf(X,n_components,alpha,beta):
    """Calculates X = WH with a Hierarchical Alternating Least Squares (HALS) Method.
    X has a lot of missing values. For now, we use masks to only deal with the entries we have.
    This
    We could later implement Expectation Maximization (discussed here: https://archive.siam.org/meetings/sdm06/proceedings/059zhangs2.pdf)
    Parameters:
        X (m x n matrix) : Matrix to be factored
        n_components (int) : # of types to find (determines dimension of W and H)
        alpha (float) : regularization term
        maxiter (int) : Maximum # of iterations
    Returns:
        W : (m x n_components matrix)
        H : (n_components x n matrix)
        converged : whether or not it converged
        
        algorithm taken from : https://arxiv.org/pdf/1401.5226.pdf
    """
    eps = 1e-5 #threshold for zero
    
    #Initialize W,H     (use clustering or svd as initial matrices?)
    W0 = np.random.rand(X.shape[0],n_components)
    W0 = np.maximum(eps,W0)
    H0 = 
    
    #SPARSIFY THAT
    
    #update W and H with HALS
    converged = False
    for i in range(max_iter):
        for r in range(n_components):
        #first we do W
            lefthand = X@H0[r,:]
            righthand = np.sum([W0[:,k] * np.dot(H0[k,:],H0[r,:]) for k in range(n_components) if k != r], axis=0)
            bottom = la.norm(H0[r,:])
            W1[:,r] = np.maximum(eps,(lefthand - righthand) / bottom)
            
        for r in range(n_components):
        #then we do H
            lefthand = W1[:,r]@X
            righthand = np.sum([H0[r,:] * np.dot(W1[:,k],W1[:,r]) for k in range(n_components) if k != r], axis=0)
            bottom = la.norm(W1[:,r])
            H1[r,:] = np.maximum(eps,(lefthand - righthand) / bottom)
        
    return W,H,converged